In [1]:
import os
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models.detection as detection
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
class dataSet(Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = [file for file in os.listdir(img_dir) if file.endswith('.jpg')]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        try:
            img_name = self.images[idx]
            img_path = os.path.join(self.img_dir, img_name)
            image = Image.open(img_path).convert("RGB")

            if self.transform:
                image = self.transform(image)

            label_path = os.path.join(self.label_dir, img_name.replace('.jpg', '.txt'))
            labels = np.loadtxt(label_path, delimiter=' ')
            if labels.ndim == 1:
                labels = labels[np.newaxis, :]

            boxes = []
            for label in labels:
                label_id, x_center, y_center, width, height = label
                x1 = (x_center - width / 2)
                y1 = (y_center - height / 2)
                x2 = (x_center + width / 2)
                y2 = (y_center + height / 2)
                boxes.append([x1, y1, x2, y2])
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels[:, 0], dtype=torch.int64)

            target = {"boxes": boxes, "labels": labels}
            return image, target
        except Exception as e:
            print(f"Error loading data for index {idx}: {e}")
            return torch.zeros(3, 139, 132), {"boxes": torch.zeros(0, 4), "labels": torch.zeros(0, dtype=torch.int64)}

In [3]:
def collate_fn(batch):
    images, targets = zip(*batch)
    return torch.stack(images), targets

In [4]:
transform = transforms.Compose([
    transforms.Resize((139, 132)),
    transforms.ToTensor()
])

In [5]:
train_img_path = os.path.join(os.getcwd(), "test", "images")
train_labels_path = os.path.join(os.getcwd(), "test", "labels")
print(train_img_path)
print(train_labels_path)


/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/test/images
/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/test/labels


In [6]:
dataset = dataSet(train_img_path, train_labels_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [7]:
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 4  # 3 classes + 1 background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

/Users/benarviv/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/benarviv/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [9]:
num_epochs = 5
for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        print(f'Loss: {losses.item()}')
    print(f'Epoch {epoch + 1}/{num_epochs} finished')

  0%|          | 0/5 [00:00<?, ?it/s]/var/folders/40/p9dn0j5n0p3372d1ncn750nm0000gn/T/ipykernel_20925/153365976.py:21: UserWarning: loadtxt: input contained no data: "/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/test/labels/volume_66_slice_135_jpg.rf.8a885cdec05966a649918a2586ff0753.txt"
  labels = np.loadtxt(label_path, delimiter=' ')
/var/folders/40/p9dn0j5n0p3372d1ncn750nm0000gn/T/ipykernel_20925/153365976.py:21: UserWarning: loadtxt: input contained no data: "/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/test/labels/volume_228_slice_30_jpg.rf.7179b33ef6dc35067dd19e043f607127.txt"
  labels = np.loadtxt(label_path, delimiter=' ')
/var/folders/40/p9dn0j5n0p3372d1ncn750nm0000gn/T/ipykernel_20925/153365976.py:21: UserWarning: loadtxt: input contained no data: "/Users/benarviv/Documents/OpenU/2024א/Data Science Workshop/Data Science Project/Brain-Tumor-Detection/test/labels/

Error loading data for index 106: not enough values to unpack (expected 5, got 0)
Error loading data for index 668: not enough values to unpack (expected 5, got 0)
Error loading data for index 661: not enough values to unpack (expected 5, got 0)
Error loading data for index 338: not enough values to unpack (expected 5, got 0)
Error loading data for index 246: not enough values to unpack (expected 5, got 0)


: 